# Feature engineering


In [7]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from trackml.dataset import load_event
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

from scripts.tracks import get_tracks
from scripts.pixels  import pixel_angles
from scripts.pixels  import cluster_size

pd.set_option('display.max_columns', None)

#### Load data

In [8]:
path2data = '/home/alepfu/Desktop/dataset/train_100_events/'
event_id = 'event000001000'
hits, cells, particles, truth = load_event(path2data + event_id)

merge = pd.merge(truth, pd.DataFrame(hits), on=['hit_id'])
hits = merge.copy()

print(hits.shape)

(120939, 15)


#### Wrangle data and get hits for a subset of tracks

In [9]:
N_tracks = 0
hits = get_tracks(hits, N_tracks=N_tracks, drop_zero_weights=False, drop_zero_id=False, factorize_particle_id=True)
print(hits.shape)

(120939, 15)


## Features from coordinates

### Add scaled cartesian coordinates

Since machine learning algorithms tend to work better with normalized features, we add the scaled coordinates to the dataset.

In [10]:
hits = hits.assign(x_scaled = hits['x'])
hits = hits.assign(y_scaled = hits['y'])
hits = hits.assign(z_scaled = hits['z'])
scaler = StandardScaler()
hits[['x_scaled', 'y_scaled', 'z_scaled']] = scaler.fit_transform(hits[['x', 'y', 'z']])

### Add cylindrical coordinates
The detector has the shape of a barrel, therefore we add [cylindrical coordinates](http://mathworld.wolfram.com/CylindricalCoordinates.html) to the dataset. The cylindrical coordinates $(r,\theta,z)$, with $r\geq 0$ and $\theta\in[0,2\pi]$, are given by $r=\sqrt{x^2+y^2}, \theta=\tan^{-1}\left(\frac{y}{x}\right)$ and $ z=z$.






In [11]:
hits = hits.assign(r_cyl= np.sqrt(hits['x']**2 + hits['y']**2))
hits = hits.assign(theta = np.arctan(np.divide(hits['y'], hits['x'])))

### Add spherical coordinates

We also add [spherical coordinates](http://mathworld.wolfram.com/SphericalCoordinates.html) to the dataset, since they strongly realte to cylindrical coordinates and in addition also account for the polar angle/distance information. Spherical coordinates $(r,\theta,\phi)$, with $r\geq 0, \theta\in[0,2\pi] $ and $ \phi\in[0,\pi]$, are given by $ r=\sqrt{x^2+y^2+z^2}, \theta=\tan^{-1}\left(\frac{y}{x}\right) $ and $ \phi=\cos^{-1}\left(\frac{z}{r}\right)$.

In [12]:
hits = hits.assign(r_sph= np.sqrt(hits['x']**2 + hits['y']**2 + hits['z']**2))
hits = hits.assign(phi = np.arccos(np.divide(hits['z'], hits['r_sph'])))

### Add atan2 features for angles

To account quadrant information for the azimuthal and polar angles, we add [atan2](https://en.wikipedia.org/wiki/Atan2) versions of $\theta$ and $\phi$.

In [13]:
hits = hits.assign(theta_atan2 = np.arctan2(hits['y'], hits['x']))
hits = hits.assign(phi_atan2 = np.arctan2(hits['z'], hits['r_sph']))

### Add radial scaled cartesian coordinates

The idea behind these features is to account for the radial displacement from spherical and cylindrical coordinates, by combining it with the cartesian coordinates and normalizing it them.

In [14]:
hits = hits.assign(x_r_cyl = hits['x']/hits['r_cyl'])
hits = hits.assign(y_r_cyl = hits['y']/hits['r_cyl'])
hits = hits.assign(z_r_cyl = hits['z']/hits['r_cyl'])
hits = hits.assign(x_r_sph = hits['x']/hits['r_sph'])
hits = hits.assign(y_r_sph = hits['y']/hits['r_sph'])
hits = hits.assign(z_r_sph = hits['z']/hits['r_sph'])
scaler = StandardScaler()
hits[['x_r_cyl','y_r_cyl','z_r_cyl','x_r_sph','y_r_sph','z_r_sph']] = scaler.fit_transform(hits[['x_r_cyl','y_r_cyl','z_r_cyl','x_r_sph','y_r_sph','z_r_sph']])

#### Display all new coordinates features

In [15]:
hits[['hit_id','x_scaled', 'y_scaled', 'z_scaled','r_cyl','r_sph','theta','theta_atan2','phi','phi_atan2',
      'x_r_cyl','y_r_cyl','z_r_cyl','x_r_sph','y_r_sph','z_r_sph']].head(1)

,hit_id,x_scaled,y_scaled,z_scaled,r_cyl,r_sph,theta,theta_atan2,phi,phi_atan2,x_r_cyl,y_r_cyl,z_r_cyl,x_r_sph,y_r_sph,z_r_sph
0,1,-0.204899,-0.039043,-1.412726,64.807045,1503.896973,0.110765,-3.030827,3.098488,-0.784934,-1.394274,-0.172688,-2.712489,-0.10494,-0.037798,-1.188751


## Features from cells

The hit coordinates from the hits file are reconstructed values from a multiplicity of activated detector cells (pixels). The quantities that were used in the hit reconstruction have therefore the possiblity to be valuable features. The cells file relates hits to pixels and allows for the computation of various quantities.

### Add features from accumlated cells information

The following features derive from the set of pixels per hit:
* Number of pixels
* Sum of signal values (the full charge deposited)

In [16]:
hit_ncells = pd.DataFrame(cells.groupby('hit_id').size()).reset_index()
hit_ncells.rename(columns={0:'n_pixel'}, inplace=True)
hits = hits.join(hit_ncells.set_index('hit_id'), on='hit_id')

hit_signal = pd.DataFrame(cells.groupby('hit_id').sum()).reset_index()
hit_signal = hit_signal[['hit_id','value']]
hit_signal.rename(columns={'value':'signal_pixel'}, inplace=True)
hits = hits.join(hit_signal.set_index('hit_id'), on='hit_id')

### Add features computed from cells information

For a set of pixel we are able to compute the following quantities:
* Pixel angles (azimuthal and polar)
* Path length through the pixel cluster
* The dimension (u,v) of the pixel cluster

In [17]:
%%time
detector = pd.read_csv('/home/alepfu/Desktop/dataset/detectors.csv')

hits['pixel_angles'] = hits.apply(lambda hit: pixel_angles(hit['x'],hit['y'],hit['z'],hit['r_sph'],
                                                           hit['volume_id'],hit['layer_id'],hit['module_id'], 
                                                           detector), axis=1)

hits[['phi_pixel', 'theta_pixel']] = hits['pixel_angles'].apply(pd.Series)
hits = hits.drop(['pixel_angles'], axis=1)

hits['cluster_size'] = hits.apply(lambda hit: cluster_size(hit['phi_pixel'],hit['theta_pixel'],
                                                           hit['volume_id'],hit['layer_id'],hit['module_id'], 
                                                           detector), axis=1)

hits[['path_pixel','u_pixel','v_pixel']] = hits['cluster_size'].apply(pd.Series)
hits = hits.drop(['cluster_size'], axis=1)

CPU times: user 9min 32s, sys: 184 ms, total: 9min 32s
Wall time: 9min 32s


#### Display all new cells features

In [18]:
hits[['hit_id','n_pixel','signal_pixel','phi_pixel','theta_pixel','path_pixel','u_pixel','v_pixel']].head(1)

,hit_id,n_pixel,signal_pixel,phi_pixel,theta_pixel,path_pixel,u_pixel,v_pixel
0,1,3,0.305441,1.538571,0.043106,0.300279,0.008338,0.229922


## Features from detector layout

### Add features for volume and layer information

Features deriving from the volumen and layer information include:
* Volume indicators (wether a hit occured in the inner/short/long parts of the detector or not)
* Feature combinations of volume and layer IDs (multiply, add and concat as integers)

In [19]:
hits['vol_inner'] = 1 * np.isin(hits.volume_id.values, [7, 8, 9])
hits['vol_short'] = 1 * np.isin(hits.volume_id.values, [12, 13, 14])
hits['vol_long'] = 1 * np.isin(hits.volume_id.values, [16, 17, 18])

hits['vol_lay_mul'] = np.multiply(hits.volume_id.values, hits.layer_id.values)
hits['vol_lay_add'] = np.add(hits.volume_id.values, hits.layer_id.values)

hits['vol_lay_concat'] = hits['volume_id'].map(str) + hits['layer_id'].map(str)
hits['vol_lay_concat'] = hits['vol_lay_concat'].astype(int)

### Add Z direction feature

Since only a small portion of tracks pass through the Z-origin, we can add a direction feature for hits belonging to particles going forward or backward.

In [20]:
min_max_z = full_merge.groupby('particle_id').agg({'z':[np.min, np.max]}).reset_index()
min_max_z.columns = ["".join(x) for x in min_max_z.columns.ravel()]
min_max_z['passes_origin'] = min_max_z.apply(lambda p: (p.zamin < 0) and (p.zamax > 0), axis=1)

print('Percentage of tracks that go through the Z-origin:',round((min_max_z.loc[min_max_z['passes_origin']].shape[0]/full_merge.shape[0])*100, 2),'%')

hits['z_dir'] = np.sign(hits['z'])

Percentage of tracks that go through the Z-origin: 0.45 %


### Add cosine distances features for ZR (cylindrical)

[Cosine distance](https://en.wikipedia.org/wiki/Cosine_similarity) measures the angle between two points on a plane. On the ZR plane tracks are straight lines and therefore the hits are aligned, which gives us the intuition to make use of a angle-based distance or similarity measure. To explore the use of cosine distance we introduce features that represent the cosine distance between ever hit and:

* the bottom left corner of the detector in the ZR plane (3000, 0)
* the bottom right corner of the detecor in the ZR plane (-3000, 0)


In [21]:
rc = np.asarray([3000,0]).reshape(1,-1)    # right bottom corner
lc = np.asarray([-3000,0]).reshape(1,-1)   # left bottom corner

rc_dists = []
lc_dists = []
for index, hit in hits.iterrows():
    zr = np.asarray([hit['z'], hit['r_cyl']]).reshape(1,-1)
    rc_dists.append(1 - cosine_similarity(rc, zr).ravel())
    lc_dists.append(1 - cosine_similarity(lc, zr).ravel())

hits['cos_dist_rc'] = np.vstack(rc_dists)
hits['cos_dist_lc'] = np.vstack(lc_dists)

#### Display all new detector layout features

In [22]:
hits[['hit_id','vol_inner','vol_short','vol_long','vol_lay_mul','vol_lay_add','vol_lay_concat',
      'z_dir','cos_dist_rc','cos_dist_lc']].head(1)

,hit_id,vol_inner,vol_short,vol_long,vol_lay_mul,vol_lay_add,vol_lay_concat,z_dir,cos_dist_rc,cos_dist_lc
0,1,1,0,0,14,9,72,-1.0,1.999071,0.000929


#### Pickle dataframe

In [23]:
hits.shape

(120939, 46)

In [24]:
hits.to_pickle('/home/alepfu/Desktop/dataset/pickle/' + event_id + '_FE.pkl')
print('done.')

done.
